In [1]:
# !sudo apt-get install python3.7
# !sudo apt-get update -y
# !sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.7 1
# !sudo update-alternatives --config python3

In [2]:
!pip install numpy
!pip install pyopenms
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 MB 9.0 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 9.3 MB/s eta 0:00:00a 0:00:01


In [3]:
import re

EXPERIMENT_NAME = "OVFF358"
# reading file
file1 = open(f'./{EXPERIMENT_NAME}.txt', 'r')
oligo_list = [line for line in file1.readlines() if re.search("^[a-zA-Z][1-9]", line)]
import csv

header = ['where\tcode\tseq\tDMT\tcomment\tlength\t']
data = [row.strip() for row in oligo_list]


with open('./oligos.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    # write the header
    writer.writerow(header)

    writer = csv.writer(f, quotechar='\"', delimiter='\n', quoting=csv.QUOTE_NONE, escapechar='\\')
    # write the data
    writer.writerow(data)

In [4]:
import pandas as pd

df = pd.read_csv('./oligos.csv', delimiter='\t')
# drop NaN seq
df = df[df['seq'].notna()]
oligo_list = df['seq'].to_list()

In [5]:
from Bio.Seq import Seq
from Bio.SeqUtils import molecular_weight

# Bio.SeqUtils.molecular_weight(seq, seq_type='DNA', double_stranded=False, circular=False, monoisotopic=False)

from pyopenms import *

# # reading file
# file1 = open('./OVFF317_oligos_seq.txt', 'r')
# oligo_list = file1.readlines()

mass_list=[]
# https://biopython.org/docs/1.75/api/Bio.SeqUtils.html
for oligo_seq in oligo_list: 
  try:
    my_dna = Seq(oligo_seq)
  except:
    print(oligo_seq)
  try:
    print(f"{oligo_seq.strip()}\t" + "%0.2f" % (molecular_weight(my_dna)-79.97))
    mass_list+=["%0.2f" % (molecular_weight(my_dna)-79.97)]
  except ValueError:
    print(f"{oligo_seq.strip()}\t" + "'N' is not a valid unambiguous letter for DNA")
    mass_list+=["NaN"]

CGA CGG GCC CGG GAT CCG ATA TCT GTA TCT TAA AGA CTA TCA TCA AGA	14760.51
CCT CGC GAA TGC ATC TAG ATT AAT CAT AGC GAA GTA ACC TTT TGA	13802.91
CGA CGG GCC CGG GAT CCG ATT CAA AAG GTT ACT TCG CTA TGA T	13243.51
CCT CGC GAA TGC ATC TAG ATA CCA TAG ATA TCT TGT GGT TTA TCA G	14114.09
CGA CGG GCC CGG GAT CCG ATC TGA TAA ACC ACA AGA TAT CTA TGG TA	14481.33
CCT CGC GAA TGC ATC TAG ATA GCA CTA ATG AAT AGT TCA CGT	12881.32
CGA CGG GCC CGG GAT CCG ATT TTA CGT GAA CTA TTC ATT AGT GC	13538.69
CCT CGC GAA TGC ATC TAG ATA TAG ACT GAT TAC TTG CAT GTT AGT	13824.91
CGA CGG GCC CGG GAT CCG ATA ACT AAC ATG CAA GTA ATC AGT CTA	13847.93
CCT CGC GAA TGC ATC TAG ATT AGG ATT GTG GAA AGA TTT CAT AGG	13938.98
CGA CGG GCC CGG GAT CCG ATC CTA TGA AAT CTT TCC ACA ATC C	13108.43
CCT CGC GAA TGC ATC TAG ATC AAA GAT TCT TAC CGA ACA TCA AA	13436.70
CGA CGG GCC CGG GAT CCG ATC TTT GAT GTT CGG TAA GAA TCT TT	13569.70
CCT CGC GAA TGC ATC TAG ATT TTC TTA TTG ACT TGA AGC TTC TCT A	14031.03
CGA CGG GCC CGG GAT CCG ATA GAG AA

In [6]:
df['monoisotopic_mass']=mass_list

In [7]:
export_dataframe=df[['where', 'code', 'length', 'seq', 'monoisotopic_mass']]

In [8]:
export_dataframe.to_csv(f'./{EXPERIMENT_NAME}_calculated.csv', sep = '\t', index=False)

In [9]:
export_dataframe


,where,code,length,seq,monoisotopic_mass
0,A1,VibriophageN4_F1-F_(pTZ/F),#= 48,CGA CGG GCC CGG GAT CCG ATA TCT GTA TCT TAA AG...,14760.51
1,B1,VibriophageN4_F1-R_(pTZ/F),#= 45,CCT CGC GAA TGC ATC TAG ATT AAT CAT AGC GAA GT...,13802.91
2,C1,VibriophageN4_F2-F_(pTZ/F),#= 43,CGA CGG GCC CGG GAT CCG ATT CAA AAG GTT ACT TC...,13243.51
3,D1,VibriophageN4_F2-R_(pTZ/F),#= 46,CCT CGC GAA TGC ATC TAG ATA CCA TAG ATA TCT TG...,14114.09
4,E1,VibriophageN4_F3-F_(pTZ/F),#= 47,CGA CGG GCC CGG GAT CCG ATC TGA TAA ACC ACA AG...,14481.33
...,...,...,...,...,...
312,I20,Alpha_1000_2_RIGHT,#= 22,CTG GAG ATA GGG ACA GGC TCA T,6824.43
313,J20,Alpha_400_1_LEFT,#= 22,TGT GGT TCT GAT GGC AAA CCT G,6781.39
314,K20,Alpha_400_1__RIGHT,#= 20,TTC CAG TCA CCC ACA CTC CA,5941.87
315,L20,Alpha_400_2_LEFT,#= 25,AGA CCT GTT TTC TTT CAT GCT CTC A,7548.89


In [10]:
molecular_weight(Seq("TG"))

651.4144

In [11]:
print([ord(char) - 96 for char in input('Write Text: ').lower()])

Write Text: 
[]


In [12]:
seq = "CCCGGGGTACCACGGAGAGATGGTGGAAATCATCTTTCTCGTGGGCATCCTTGATGGCCACCTCGTCGGAAGTGCCCACGAGGATGACAGCAATGCCAATGCTGGGGGGGCTCTTCTGAGAACGAGCTCTGCTGCCTGACACGGCCAGGACGGCCAACACCAACCAGAACTTGGGAGAACAGCACTCCGCTCTGGGCTTCATCTTCAACTCGTCGACTCCCTGCAAACACAAAGAAAGAGCATGTTAAAATAGGATCTACATCACGTAACCTGTCTTAGAAGAGGCTAGATACTGCAATTCAAGGACCTTATCTCCTTTCATTGAGCACCAAACCCAACTCCATCTACCAGCCTACTCTCTTATCTCTGGTATT"


In [13]:
seq.upper()

'CCCGGGGTACCACGGAGAGATGGTGGAAATCATCTTTCTCGTGGGCATCCTTGATGGCCACCTCGTCGGAAGTGCCCACGAGGATGACAGCAATGCCAATGCTGGGGGGGCTCTTCTGAGAACGAGCTCTGCTGCCTGACACGGCCAGGACGGCCAACACCAACCAGAACTTGGGAGAACAGCACTCCGCTCTGGGCTTCATCTTCAACTCGTCGACTCCCTGCAAACACAAAGAAAGAGCATGTTAAAATAGGATCTACATCACGTAACCTGTCTTAGAAGAGGCTAGATACTGCAATTCAAGGACCTTATCTCCTTTCATTGAGCACCAAACCCAACTCCATCTACCAGCCTACTCTCTTATCTCTGGTATT'

In [ ]:
len(seq)
my_dna = Seq(seq)
molecular_weight(my_dna)